In [53]:
#|default_exp generative_callback

In [54]:
#|hide
%reload_ext autoreload
%autoreload 2

In [55]:
#|hide
from nbdev.showdoc import show_doc

In [56]:
#|export
from fastai.callback.core import Callback
from fastai.text.all import *
from typing import List
from fastinference.inference import *
from denovo_design.generative_basics import *
from denovo_design.chem import is_valid
from fcd_torch import *
from random import choices
from guacamol.distribution_learning_benchmark import ValidityBenchmark, UniquenessBenchmark, NoveltyBenchmark
from guacamol.frechet_benchmark import FrechetBenchmark
from guacamol.distribution_matching_generator import DistributionMatchingGenerator

In [57]:
#|export    
class MockGenerator(DistributionMatchingGenerator):
    """
    Mock generator that returns pre-defined molecules,
    possibly split in several calls
    """

    def __init__(self, molecules) -> None:
        self.molecules = molecules
        self.cursor = 0

    def generate(self, number_samples: int):
        end = self.cursor + number_samples

        sampled_molecules = self.molecules[self.cursor:end]
        self.cursor = end
        return sampled_molecules

In [58]:
#|export
class SMILESLogger:
    def __init__(self, fname:str='generated_molecules.smi',mode:str='a+'):
        self.fname = fname
        self.mode = mode
        self.output_file = open(fname, mode, buffering=1)
            
    def write(self, text:str=''):
        self.output_file.write(text + '\n')
        
    def close(self):
        self.output_file.close()

In [73]:
#|export
class GenerativeCallback(Callback):
    
    """
    
    A callback to generate molecules while training a model. 

    
    """

    def __init__(self, reference_mols:List=[], text:str='', max_size:int=100, temperature:float=0.7, max_mols:int=100, output_file:str='generated_mols.smi'):
        
        """
        Arguments:
        
            reference_mols : list
                A list of reference SMILES to compute validation metrics
                
            text : str
                Seed string (default = '')
                
            max_size : int
                Maximum size of generate SMILES 
                
            temperature : float
                Sampling temperature (default = 1.0)
                
            max_mols : int
                Number of molecules to generate
                
            output_file : str
                A text file where SMILES will be saved after each epoch.
                
        Returns:
        
            decoded : list
                A list of generated SMILES.
        
        """
        
        super().__init__()
        self.reference_mols = reference_mols
        self.text = text
        self.max_size = max_size
        self.temperature = temperature
        self.max_mols = max_mols
        self.smiles = []
        self.valid_mols = []
        self.epoch_track = 0
        self.output_file = output_file
        
        # Define the benchmark before training because it needs to calculate the mean and covariance for ref mols
        self.fcd_benchmark = FrechetBenchmark(training_set=reference_mols, sample_size=len(reference_mols))
                    
 
    def sampling(self):

        self.model.reset()    # Reset the model

        nums = self.dls.numericalize
        stop_index = self.dls.train.vocab.index('xxbos')

        idxs = idxs_all = self.dls.test_dl([self.text]).items[0].to(self.dls.device)
        for _ in range(self.max_size):
            preds = self.get_preds(dl=[(idxs[None],)], decoded_loss=False)

            res = torch.tensor(preds[0][0][-1])
            if self.temperature != 1.: res.pow_(1 / self.temperature)
            idx = torch.multinomial(res, 1).item()
            if idx != stop_index:

                idxs = idxs_all = torch.cat([idxs_all, idxs.new([idx])])
            else:
                break
        decoded = ''.join([nums.vocab[o] for o in idxs_all if nums.vocab[o] not in [BOS, PAD]])  # Decode predicted tokens
        return decoded
    

    def _validity_score(self):
        gen = MockGenerator(self.smiles)
        val = ValidityBenchmark(number_samples=len(gen.molecules)).assess_model(gen).score        
        return val
    
    def _fcd_score(self):
        gen = MockGenerator(self.smiles)
        fcd_score = self.fcd_benchmark.assess_model(gen).score
        return fcd_score

    def _uniqueness_score(self): 
        gen = MockGenerator(self.smiles)
        unq = UniquenessBenchmark(number_samples=len(gen.molecules)).assess_model(gen).score
        return unq

    def _novelty_score(self):
        gen = MockGenerator(self.smiles)
        nov = NoveltyBenchmark(number_samples=len(gen.molecules),training_set=self.reference_mols).assess_model(gen).score
        return nov
    
    def before_fit(self):
        self.logger = SMILESLogger(self.output_file)
        
    def before_epoch(self):
        self.val, self.unq, self.nov = 0, 0, 0
        self.smiles = []
        self.valid_mols = []
        
    def after_epoch(self):
        self.epoch_track += 1

    def before_validate(self, **kwargs):
        self.smiles += [self.sampling() for _ in range(self.max_mols)]
        _valid_smiles = list(filter(is_valid, self.smiles))
        for smi in _valid_smiles:
            self.logger.write(f'{self.epoch_track},{smi}')
        
    def after_fit(self, **kwargs):
        self.logger.close()

In [74]:
show_doc(GenerativeCallback)

---

### GenerativeCallback

>      GenerativeCallback (reference_mols:List=[], text:str='',
>                          max_size:int=100, temperature:float=0.7,
>                          max_mols:int=100,
>                          output_file:str='generated_mols.smi')

A callback to generate molecules while training a model. 

In [75]:
#| hide
from nbdev import nbdev_export
nbdev_export()